# OpenStreetMap Project - Data Wrangling with SQL
---

### Table of Contents

* [Project Description](#Project-Description)
* [Map Area](#Map-Area)
* [Problems Encountered in the Map](#Problems-Encountered-in-the-Map)  
    * [Incosistent or overlapping building types](#Incosistent-or-overlapping-building-types)     
    * [Incosistent phone numbers](#Incosistent-phone-numbers)    
* [Data Overview](#Data-Overview)
* [Additional Ideas](#Additional-Ideas)
* [References](#References)


### Project Description
---
[OpenStreetMap](https://www.openstreetmap.org) is a free, editable map of the whole world that is being built by volunteers largely from scratch and released with an open-content license.  
The [OpenStreetMap License](https://www.openstreetmap.org/copyright) allows free (or almost free) access to our map images and all of our underlying map data. The project aims to promote new and interesting uses of this data.

The data is stored in XML format and is generated by different users and therefore prone to many errors.

In this Project, I will choose a part of the world and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap.

Finally, I will use SQL to  store, query, and aggregate the collected data.

### Map Area
---
<img src="Metro-Extracts_Warsaw.png" alt="Metro-Extracts_Warsaw" style="width: 400px;"/>

[Warsaw, Poland](https://mapzen.com/data/metro-extracts/metro/warsaw_poland/)

This map is from the town I probably spend most of my time in and am quite fond of.  I’m quite interested to see what database querying reveals, and I’d like an opportunity to contribute to its improvement on OpenStreetMap.org.

It is the suggested Metro Extract and the overall filesize of the unzipped xml file is ~77mb.

### Problems Encountered in the Map
---
After initially downloading a smaller sample size of the Warsaw I started
the wrangling process by investigating the various elements.


I did not find  any problems with street abreviations as it often happens because anly th name of the street is used.
In cases of square the description plac is correctly used.

The problems encountered, and I will discuss are the following:

* Incosistent or overlapping building types ('boat', 'boathouse','houseboat' or 'civ','civic')

* Incosistent phone numbers(+ 48 22 620, 00482263210, 22 31 82 80 )




### Incosistent or overlapping building types

Looking into the building 'key' I noticed that there are many custom entries that do not appear at the wiky for this 'key'.

I chose to use the json provided by the [API](https://taginfo.openstreetmap.org/taginfo/apidoc#api_4_key_values) with the most common values and filter the values of my map based on their number of occurences or the existence of a wiki:

The number of `building` values that did not have a wiki entry and had less than 5 occurences in the OSM db where 59.

I audited them using the following mapping: 

    building_mapping={}

    building_mapping['day_care'] = 'daycare'
    building_mapping['Basen'] = 'swimming_pool'
    building_mapping['family_house'] = 'house'
    building_mapping['classrooms'] = 'school'
    building_mapping['family_house'] = 'house'

    for val in audit_building:
        if  'shop' in val or 'store' in val or 'retail' in val or 'food' in val:
            building_mapping[val] = 'retail'
        if 'office' in val or 'commercial' in val:
            building_mapping[val] = 'commercial'
        if 'residential' in val:
            building_mapping[val] = 'apartments'
        if 'museum' in val:
            building_mapping[val] = 'museum'
        if 'ruin' in val or 'collapsed' in val:
            building_mapping[val] = 'ruins'
        if 'terrac' in val:
            building_mapping[val] = 'terrace'    
        if 'serv' in val or 'power' in val:
            building_mapping[val] = 'service'  
        if 'avia' in val:
            building_mapping[val] = 'hangar' 
        if 'mbass' in val:
            building_mapping[val] = 'embassy'
        if 'boat' in val:
            building_mapping[val] = 'houseboat'
        if 'otel' in val:
            building_mapping[val] = 'hotel'
        if 'detache' in val:
            building_mapping[val] = 'detached'  
        if  'clini' in val or 'docto' in val:
            building_mapping[val] = 'hospital'
        if  'convent' in val or 'basilica' in val or 'monastery' in val :
            building_mapping[val] = 'church'
        if  'factory' in val :
            building_mapping[val] = 'industrial'
        if  'shrine' in val :
            building_mapping[val] = 'shrine'
        if  'glass' in val :
            building_mapping[val] = 'conservatory'
        if  'enter' in val or 'hall' in val:
            building_mapping[val] = 'civic' 
        
The following keys: `('belfry','cage','castle','corridor','empty','enclosing','fast_food','fuel','library','military','monument',
'no','palace','part','passage','prison','silo','storage_tank','tent','terminal','tower', 'tribunes')`    
where not mapped since I could not find an adequate category for these without being in danger of deleting precious information. 

### Incosistent phone numbers

Looking into the phone number 'key' where quite messy and incosistent.

According to the [Key:phone - wiki](http://wiki.openstreetmap.org/wiki/Key:phone) the phone numbers should have the following format:

`phone=+<country code> <area code> <local number>`, following the ITU-T E.123 and the DIN 5008 pattern

I used the function in audit. py to transform them programatically to the desired format taking into consideration the possibility that foreign numbers may exist. Multible numbers where ommited.

### Data Overview

### Additional Ideas

### Referennces

In [13]:
.exit

Restarting SQLite3

In [14]:
.mode csv
.import Warsaw_Small_ways.csv ways

In [20]:
SELECT count(DISTINCT timestamp) FROM ways 

24718


In [21]:
.mode csv
.import Warsaw_Small_ways_tags.csv ways_tags

In [22]:
.schema ways_tags

CREATE TABLE ways_tags(
  "id" TEXT,
  "key" TEXT,
  "value" TEXT,
  "type" TEXT
);


In [42]:
.mode columns
.headers on

In [43]:
SELECT DISTINCT type, count(type) as ct FROM ways_tags GROUP BY type ORDER BY ct 

type        ct        
----------  ----------
abandoned   1         
change      1         
disease     1         
franchise   1         
healthcare  1         
medical_sy  1         
monument    1         
official_n  1         
recycling   1         
short_name  1         
tower       1         
aircraft    2         
health_fac  2         
health_spe  2         
maxgcweigh  2         
maxheight   2         
old_addr    2         
old_name    2         
rmikke      2         
was         2         
demolished  3         
informatio  3         
mtb         3         
old         3         
alt_name    4         
generator   4         
is_in       4         
bicycle     5         
internet_a  5         
parking     5         
payment     5         
bridge      6         
historic    6         
ramp        6         
taxi        6         
social_fac  7         
toilets     7         
maxgvweigh  8         
maxweight   8         
access      10        
heritage    10        
ref        

In [44]:
SELECT DISTINCT key, count(key) as ct FROM ways_tags GROUP BY key ORDER BY ct 

key         ct        
----------  ----------
2           1         
FIXME       1         
Nowych      1         
accor_tick  1         
addr:postc  1         
alternativ  1         
amenity_2   1         
autism      1         
be          1         
beds        1         
biodiesel   1         
boat        1         
building:l  1         
building_1  1         
delivery    1         
dentistry   1         
dog         1         
factory     1         
female      1         
full_name   1         
function    1         
golf        1         
he          1         
hoops       1         
housename:  1         
icao        1         
id          1         
ko          1         
kr          1         
leaf_cycle  1         
levels:und  1         
license_cl  1         
local_oper  1         
luggage     1         
maestro     1         
male        1         
method      1         
mooring     1         
name:uk     1         
name_1      1         
name_2      1         
octane_100 

In [41]:
SELECT DISTINCT value, count(value) as ct FROM ways_tags WHERE key='building' GROUP BY value ORDER BY value 

apartments,474
apartments;commercial,1
aviary,7
belfry,1
boat,2
boathouse,5
cage,1
car_shop,1
cathedral,1
chapel,4
church,55
civ,1
civic,3
collapsed,3
commercial,129
construction,32
container,6
convent,1
corridor,1
doctors,1
dormitory,11
enclosing,20
factory,4
garage,187
garages,314
greenhouse,5
hall,89
hangar,6
hospital,35
hotel,7
house,180
houseboat,1
hut,1
industrial,163
kindergarten,19
kiosk,1
library,1
manufacture,3
monastery,2
monument,1
no,2
office,204
office;residential,2
offices,67
palace,13
part,4
power,3
public,17
residential,2400
residential;office,1
residential;retail,1
restaurant,1
retail,227
roof,208
ruins,5
school,131
service,53
shed,59
sports_centre,7
squat,2
stadium,2
supermarket,4
synagogue,1
tent,2
terrace,45
terraces,2
train_station,11
transportation,27
university,38
utility,1
wall,2
warehouse,3
yes,12061


In [32]:
SELECT DISTINCT value, count(value) as ct FROM ways_tags WHERE key='highway' GROUP BY value ORDER BY value 

abandoned,1
bus,9
bus_stop,10
construction,70
corridor,8
crossing,30
cycleway,722
elevator,60
emergency,271
footway,13771
island,1
junction,3
living_street,414
no,3
outdoor_seating,2
parking_space,4
path,714
pedestrian,327
permissive,19
platform,378
primary,68
primary_link,18
prohibited,30
proposed,82
raceway,2
residential,1744
road,4
secondary,1739
secondary_link,260
service,5536
steps,1288
taxi,1
taxi_stop,1
tertiary,1244
tertiary_link,65
track,188
traffic_island,253
trunk,38
trunk_link,9
unclassified,86
yes,86


In [33]:
SELECT DISTINCT value, count(value) as ct FROM ways_tags WHERE key='addr' GROUP BY value ORDER BY value 

http://www.teatrwarsawy.pl/,1
http://www.zus.pl/default.asp?p=2&id=1866&kod=443,1
mapa.um.warszawa.pl,11125
mapa.um.waw.pl,3
msi,17
survey,3


In [40]:
SELECT DISTINCT value, count(value) as ct FROM ways_tags WHERE key='name' GROUP BY value ORDER BY value 

"""Bazarek przy Duracza""",1
"""Kamienna""",1
"""Koleje Mazowieckie - KM"" sp. z o.o.",1
"1 Dywizji Grenadierów-Francja 1940",1
"1 Minute",1
"10 Warszawski Pułk Samochodowy im. majora Stefana Starzyńskiego",1
"11 Listopada",18
"11 Listopada 01",1
"11 Listopada 02",1
"110kV Bemowo - Koło",1
"110kV EC Kawęczyn - Targówek",1
"110kV EC Siekierki - Batory",1
"110kV EC Żerań - Gdańska",1
"110kV EC Żerań - Targówek",1
"110kV Gdańska - Młynów",1
"110kV Gdańska - Powiśle",1
"110kV Jelonki - Koło",1
"110kV Kaliszówka - Bemowo",1
"110kV Mirów - Pałac",1
"110kV Mory - Ochota/Południowa",3
"110kV Mory - ZMW",1
"110kV Mory - ZMW / Jelonki - Koło",1
"110kV Mory - Zachodnia/Wola",1
"110kV Powiśle - Pałac",1
"110kV Towarowa - Mirów",1
"110kV Towarowa - Młynów",1
"110kV Towarowa - Śródmieście",1
"110kV Wola - Słodowiec",4
"110kV Wschodnia - FSO",1
"110kV Wschodnia - FSO / EC Kawęczyn - Targówek",1
"110kV Wschodnia - FSO / EC Żerań - Targówek",1
"110kV Śródmiescie - Batory",1
"19. Dzielnica",1
"2 Społecz

In [2]:
.mode csv
.import Warsaw_Small_nodes_tags.csv nodes_tags

In [1]:
# .mode columns
# .headers on

Error: unrecognized token: "#"


In [3]:
SELECT DISTINCT key, count(key) as ct FROM nodes_tags GROUP BY key ORDER BY ct 

key         ct        
----------  ----------
1           1         
NFZ         1         
ab          1         
abbr_name   1         
ace         1         
acupunctur  1         
admin_leve  1         
aeroway     1         
af          1         
ak          1         
am          1         
american_e  1         
amex        1         
an          1         
ang         1         
animated    1         
ar          1         
arc         1         
arz         1         
ast         1         
automated   1         
autor       1         
az          1         
ba          1         
bancomat    1         
bat-smg     1         
batteries   1         
be          1         
be-tarask   1         
beer        1         
bg          1         
bi          1         
biogas      1         
bn          1         
bo          1         
br          1         
bs          1         
bunker_typ  1         
ca          1         
capital     1         
child       1         
ciag_ulicy 

In [5]:
SELECT DISTINCT value, count(value) as ct FROM nodes_tags WHERE key='phone' GROUP BY value ORDER BY value 

value        ct        
-----------  ----------
(22)6280023  1         
+ 48 22 620  1         
+ 48 22 841  1         
+4622252185  1         
+48  22 211  1         
+48 (22) 53  1         
+48 (22) 59  1         
+48 22 121   1         
+48 22 123   1         
+48 22 185   1         
+48 22 211   1         
+48 22 225   1         
+48 22 2346  1         
+48 22 240   1         
+48 22 2432  1         
+48 22 2529  1         
+48 22 258   1         
+48 22 258   1         
+48 22 278   1         
+48 22 314   1         
+48 22 321   1         
+48 22 321   1         
+48 22 331   1         
+48 22 349   1         
+48 22 400   1         
+48 22 404   1         
+48 22 413   1         
+48 22 416   1         
+48 22 418   1         
+48 22 436   1         
+48 22 44 3  1         
+48 22 460   1         
+48 22 462   1         
+48 22 4648  1         
+48 22 487   1         
+48 22 490   1         
+48 22 505   1         
+48 22 520   1         
+48 22 53 9  1         
+48 22 531   1  

In [2]:
SELECT DISTINCT type, count(type) as ct FROM nodes_tags GROUP BY type ORDER BY ct 

type           ct        
-------------  ----------
communication  1         
flag           1         
health_facili  1         
heritage       1         
historic       1         
inscritpion    1         
monitoring     1         
provided_for   1         
ref            1         
swing_gate     1         
website        1         
3dr            2         
ata_delete     2         
billiards      2         
drink          2         
location       2         
roof           2         
rwetempchargi  2         
whc            2         
healthcare     3         
helicopter     3         
monument       3         
opening_hours  3         
post_box       3         
species        3         
dance          4         
disabled       4         
inscription    4         
language       4         
wheelchair     4         
building       5         
health_specia  5         
description    6         
rmikke         6         
tower          6         
memorial       7         
internet_acc

In [3]:
SELECT DISTINCT value, count(value) as ct FROM nodes_tags WHERE type='addr' GROUP BY value ORDER BY value; 

00-001,2
00-002,2
00-003,2
00-004,2
00-005,2
00-006,2
00-008,4
00-009,2
00-010,2
00-011,2
00-012,4
00-013,4
00-014,4
00-018,2
00-019,6
00-020,16
00-021,12
00-022,4
00-023,34
00-024,14
00-025,2
00-026,2
00-028,4
00-029,10
00-030,2
00-031,10
00-032,2
00-033,10
00-034,2
00-036,2
00-038,6
00-039,4
00-041,4
00-042,6
00-046,2
00-048,6
00-052,6
00-054,2
00-056,2
00-057,2
00-058,2
00-067,2
00-070,6
00-071,22
00-072,4
00-073,12
00-074,2
00-075,2
00-076,16
00-077,2
00-078,4
00-080,2
00-082,8
00-085,2
00-087,6
00-094,8
00-095,10
00-097,4
00-098,4
00-099,4
00-100,2
00-102,6
00-103,2
00-105,14
00-108,2
00-110,2
00-112,2
00-113,6
00-114,4
00-116,80
00-118,2
00-123,2
00-124,12
00-127,4
00-128,2
00-131,2
00-132,22
00-133,6
00-136,2
00-138,12
00-139,6
00-140,14
00-141,12
00-142,2
00-143,2
00-144,4
00-148,26
00-150,2
00-152,4
00-157,2
00-159,14
00-164,2
00-172,8
00-173,2
00-175,10
00-180,2
00-189,4
00-193,8
00-194,2
00-199,2
00-201,2
00-203,2
00-204,2
00-213,2
00-215,4
00-219,24
00-220,12
00-223,2
00-22

In [ ]:


* Inconsistent postal codes (“NC28226”, “28226­0783”, “28226”)

* “Incorrect” postal codes (Charlotte area zip codes all begin with “282” however a large portion of all documented zip codes were outside this region.)

* Second­ level “k” tags with the value "type"(which overwrites the element’s previously processed node[“type”]field).

* Street names in second ­level “k” tags pulled from Tiger GPS data and divided into segments, in the following format: